In [43]:
import pandas as pd
import numpy as np
import pickle as p

In [44]:
data_all = pd.read_csv('data_all.csv')

# Test with Subset of Data
np.random.seed(41)
item_subset = list(np.random.choice(data_all['Item_ID'].unique(), size=5, replace=False))
data_subset = data_all.loc[data_all['Item_ID'].isin(item_subset)].copy()

# # Test with All Data
# data_subset = data_all.copy()

In [45]:
item_subset

[31, 2259, 1203, 5073, 102]

### `/train`

In [46]:
import requests
import json

url = 'http://localhost:5000/train/'

data = data_subset.to_json()

payload = {'data': data,
           'cv_acc': True,
           'project_id': 1
           }

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)

In [47]:
r.json()

{'cv_acc': {'avg_sales': {'0': 45.21694564819336,
   '1': 176.6056671142578,
   '2': 198.34840393066406,
   '3': 97.1537094116211,
   '4': 115.3660659790039},
  'item_id': {'0': 31.0, '1': 102.0, '2': 1203.0, '3': 2259.0, '4': 5073.0},
  'mae_score': {'0': 13.472270488739014,
   '1': 55.9597282409668,
   '2': 40.02078819274902,
   '3': 25.948142051696777,
   '4': 44.89441967010498},
  'mpe_score': {'0': 0.2979473800278082,
   '1': 0.31686258518965177,
   '2': 0.20177015493776773,
   '3': 0.26708338990701447,
   '4': 0.38914753042090866},
  'r2_score': {'0': 0.6170428283043433,
   '1': 0.3591472636852114,
   '2': 0.61731508452739,
   '3': 0.5268625825958735,
   '4': 0.22047645970878907},
  'rmse_score': {'0': 20.155343055725098,
   '1': 78.80782413482666,
   '2': 55.27177333831787,
   '3': 33.64593315124512,
   '4': 59.27281665802002}},
 'result': 'Success'}

In [49]:
# CV Accuracies Returned
cv_accuracies = pd.DataFrame.from_dict((r.json()['cv_acc']))
cv_accuracies

,avg_sales,item_id,mae_score,mpe_score,r2_score,rmse_score
0,45.216946,31.0,13.472270,0.297947,0.617043,20.155343
1,176.605667,102.0,55.959728,0.316863,0.359147,78.807824
2,198.348404,1203.0,40.020788,0.201770,0.617315,55.271773
3,97.153709,2259.0,25.948142,0.267083,0.526863,33.645933
4,115.366066,5073.0,44.894420,0.389148,0.220476,59.272817


### `/optimize`

In [ ]:
url = 'http://localhost:5000/optimize/'

payload = {'project_id': 1,
           'constraints': [],
           'population': 100,
           'max_epoch': 200
           }

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)

In [ ]:
best_prices = r.json()['result']
best_prices

In [ ]:
import requests
import json

url = 'http://localhost:5000/predict/'
payload = {'prices': best_prices,
           'project_id': 1
           }
headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }
payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)

# What If Prediction
qty = r.json()['qty_estimates']

In [ ]:
qty

In [ ]:
# Calculate Revenue
revenue = 0
for item in best_prices.keys():
    revenue += (best_prices[item]*qty['Qty_'+item])
    
print(revenue)

### `/predict/`

In [ ]:
# Sample Actual Price 
from helper_functions import *

sales_data = data_subset

sales_data = optimize_memory(sales_data)

sales_data_wide = sales_data.set_index(
    ['Wk', 'Tier', 'Store', 'Item_ID']).unstack(level=-1).reset_index().copy()
sales_data_wide.columns = [
    ''.join(str(i) for i in col).strip()
    for col in sales_data_wide.columns.values
]
sales_data_wide = sales_data_wide.sort_values(
    ['Tier', 'Store', 'Wk'], ascending=True).reset_index(drop=True)

sales_data_wide_clean = sales_data_wide.dropna(axis=0).copy()
dataset = sales_data_wide_clean
price_columns = [
    col for col in sales_data_wide_clean.columns if col.startswith('Price')
]

a = dataset.sample(1)
a_dict = a[price_columns].iloc[0].to_dict()
a_input = {}
for key in a_dict.keys():
    a_input[key.split('_')[1]]=round(a_dict[key],2)

print('Input Prices Quantities:')
a_input

In [ ]:
# Predict Sales Qty and Compare to Actual

import requests
import json

url = 'http://localhost:5000/predict/'

payload = {'prices': a_input, 'project_id': 1}

headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}

payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)
qty = r.json()['qty_estimates']
qty_outp = {}
for key in qty.keys():
    qty_outp[key] = round(qty[key], 0)
actual = a[[
    col for col in sales_data_wide_clean.columns if col.startswith('Qty_')
]].iloc[0].to_dict()
pd.DataFrame([qty_outp, actual], index=['Estimated', 'Actual']).transpose()